# Make Sense of Status and Settings

## What Next

- Take photos at regular intervals
- Maintain database of local photos/videos
- Download all files to local storage
- Delete files on camera
- Establish essential camera settings
    - Exposure sensitivity
    - shutter time
    - white balance, protune, etc.
- Multiple, simultaneous comm sessions?
    - Take photos at precise intervals
    - Batch download/erase photos
    - Check camera status


In [1]:
import os
import numpy as np
import wireless

import context

import gopro_helper as gopro

from gopro_helper import api
from gopro_helper import get
from gopro_helper import json_io
from gopro_helper import namespace

# JSON API

## display hints
- don't care

## status
- refers specifically to 'status' object from status GET
- associate field names with ID numbers
- categorized by informative groups (system, app, photo, wireless, storage

## modes
- modes and submodes
- other important settings like fps, resolution, etc.
- entire subsections for video, photo, audio

## services
- URLs
- list media files, only one I care about
- start/stop streaming
- firmware stuff
- auth stuff

## commands
- define URLs for issuing commands to camera
- says nothing about any additional parameters
- some duplicates with services

## filters
- I think this is used to grey out or disable certain GUI elements based on current settings
- don't care


## camera mode map
- some information on making sense of modes and submodes and their aliases

In [2]:
# Load JSON API information
api_details = json_io.read(api.fname_json)

In [3]:
# Example status group details
api_details['status']['groups'][0]

{'fields': [{'id': 1, 'name': 'internal_battery_present'},
  {'id': 2, 'name': 'internal_battery_level'},
  {'id': 3, 'name': 'external_battery_present'},
  {'id': 4, 'name': 'external_battery_level'},
  {'id': 6, 'name': 'system_hot'},
  {'id': 8, 'name': 'system_busy'},
  {'id': 9, 'name': 'quick_capture_active'},
  {'id': 10, 'name': 'encoding_active'},
  {'id': 11, 'name': 'lcd_lock_active'},
  {'id': 45, 'name': 'camera_locate_active'},
  {'id': 57, 'name': 'current_time_msec'},
  {'id': 60, 'name': 'next_poll_msec'},
  {'id': 61, 'name': 'analytics_ready'},
  {'id': 62, 'name': 'analytics_size'},
  {'id': 63, 'name': 'in_contextual_menu'},
  {'id': 68, 'name': 'gps_status'},
  {'id': 70, 'name': 'internal_battery_percentage'},
  {'id': 74, 'name': 'acc_mic_status'}],
 'group': 'system'}

In [4]:
# Example status group details
api_details['status']['groups'][1]

{'fields': [{'id': 43, 'name': 'mode'},
  {'id': 44, 'name': 'sub_mode'},
  {'id': 71, 'name': 'video_selected_flatmode'},
  {'id': 72, 'name': 'photo_selected_flatmode'},
  {'id': 73, 'name': 'timelapse_selected_flatmode'}],
 'group': 'app'}

# Get and Display Camera Info

In [5]:
# camera status GET returns two objects: 'status' and 'settings'.

content = get(api.url_status)

camera_status = content['status']
camera_settings = content['settings']

In [6]:
def entry_option_value(entry, index):
    for option in entry['options']:
        if option['value'] == index:
            # Return nice text value
            return option['display_name']
        
    # Fallback to simply returning the index number
    return str(index)


def parse_mode_values(camera_settings, api_details):
    """Associate camera numerical setting values with human-readable text.
    Store in mode-structured dict.
    """
    # Main loop over top-level modes
    info = {}
    modes_include = ['Video', 'Photo', 'Setup']   # 'Multishot', 'Audio', 'Playback', 'Broadcast']
    for mode in api_details['modes']:
        if mode['display_name'] in modes_include:

            info_mode = {}
            for entry in mode['settings']:

                # Find corresponding entry and value in camera settings output
                id_str = str(entry['id'])
                index = camera_settings[id_str]
                value = entry_option_value(entry, index)

                info_mode[entry['display_name']] = value

            info[mode['display_name']] = info_mode

    # Done
    return info


def parse_status_values(camera_status, api_details):
    """Extact a few non-retarded bits of information from 'status' group.  Store in flat dict.
    """
    # known groups:  ['system', 'storage', 'broadcast', 'fwupdate', 'liveview', 'setup', 'stream']
    groups_include = ['system', 'storage', 'app']

    info = {}
    for group in api_details['status']['groups']:
        # Check for group name in set of to-be-extracted details
        group_name = group['group']
        if group_name in groups_include:
            for entry in group['fields']:
                id_str = str(entry['id'])
                value = camera_status[id_str]

                info[entry['name']] = value
                
    # Done
    return info

In [7]:
def _pretty_status(info):
    keys = ['system_hot', 'system_busy', 'current_time_msec',
            'internal_battery_percentage', 'remaining_photos', 'remaining_video_time',
            'remaining_space', 'remaining_timelapse_time', 'num_total_photos', 'num_total_videos']
    
    info_out = {}
    for k in keys:
        k_out = k.replace('_', ' ')
        k_out = ' '.join([s.capitalize() for s in k_out.split()])
        info_out[k_out] = info[k]
        
    return info_out


def _pretty_modes(info):
    keys = {'Video': ['Color', 'White Balance', 'EV Comp', 'Field of View', 'ISO', 'ISO Mode',
                      'Low Light', 'Frames Per Second', 'Shutter', 'Resolution',
                      'Video Stabilization', 'Sharpness', 'Protune'],
            'Photo': ['Color', 'EV Comp', 'ISO MIN', 'ISO MAX', 'WDR', 'Shutter', 'Megapixels',
                      'RAW', 'Protune', 'White Balance', 'Sharpness'],
            'Setup': ['Current Flat Mode', 'Auto Off', 'GPS']}
    
    info_out = {}
    
    for n, g in keys.items():
        info_out[n] = {}
        for k in g:
            info_out[n][k] = info[n][k]
            
    return info_out

In [13]:
def get_settings(pretty=True):
    """Fetch status and mode settings information from camera.  Optionally return as nicely-
    formatted dict for useful subset.
    """
    # Fetch info from camera.  Camera status GET returns two objects: 'status' and 'settings' (aka modes).
    content = get(api.url_status)

    camera_status = content['status']
    camera_settings = content['settings']

    # Load JSON API information
    api_details = json_io.read(api.fname_json)

    # Parse status and settings details
    info_status = parse_status_values(camera_status, api_details)
    info_modes = parse_mode_values(camera_settings, api_details)
    mode = info_status['mode']
    
    if pretty:
        info_status = _pretty_status(info_status)
        info_modes = _pretty_modes(info_modes)

    _VIDEO_MODE = 0
    _PHOTO_MODE = 1
    _MULTI_MODE = 2

    # Exclude non-current mode info
    if mode == _VIDEO_MODE:
        info_modes.pop('Photo')
    elif mode == _PHOTO_MODE:
        info_modes.pop('Video')
    else:
        pass
    
    # Combine
    info_modes['System'] = info_status
    
    # Done
    return info_modes

# Example function calls

In [14]:
get_settings()

{'Setup': {'Auto Off': '30 MIN', 'Current Flat Mode': 'Video', 'GPS': 'ON'},
 'System': {'Current Time Msec': 296103,
  'Internal Battery Percentage': 91,
  'Num Total Photos': 8448,
  'Num Total Videos': 6,
  'Remaining Photos': 6317,
  'Remaining Space': 42040064,
  'Remaining Timelapse Time': 0,
  'Remaining Video Time': 4765,
  'System Busy': 0,
  'System Hot': 0},
 'Video': {'Color': 'GoPro',
  'EV Comp': '0.0',
  'Field of View': 'Wide',
  'Frames Per Second': '60',
  'ISO': '400',
  'ISO Mode': 'Max',
  'Low Light': 'ON',
  'Protune': 'ON',
  'Resolution': '2.7K',
  'Sharpness': 'Low',
  'Shutter': '1/60',
  'Video Stabilization': 'OFF',
  'White Balance': '4800K'}}

In [15]:
parse_status_values(camera_status, api_details)

{'acc_mic_status': 0,
 'analytics_ready': 2,
 'analytics_size': 0,
 'camera_locate_active': 0,
 'current_time_msec': 72785,
 'encoding_active': 0,
 'external_battery_level': 0,
 'external_battery_present': 0,
 'gps_status': 1,
 'in_contextual_menu': 0,
 'internal_battery_level': 3,
 'internal_battery_percentage': 94,
 'internal_battery_present': 1,
 'last_hilight_time_msec': 0,
 'lcd_lock_active': 0,
 'mode': 0,
 'next_poll_msec': 500,
 'num_group_photos': 32,
 'num_group_videos': 6,
 'num_hilights': 0,
 'num_total_photos': 8448,
 'num_total_videos': 6,
 'photo_selected_flatmode': 17,
 'quick_capture_active': 0,
 'remaining_photos': 6317,
 'remaining_space': 42040064,
 'remaining_timelapse_time': 0,
 'remaining_video_time': 4765,
 'sd_status': 0,
 'sub_mode': 0,
 'system_busy': 0,
 'system_hot': 0,
 'timelapse_selected_flatmode': 20,
 'video_selected_flatmode': 12}

In [16]:
parse_mode_values(camera_settings, api_details)

{'Photo': {'Color': 'GoPro',
  'Default Photo Sub Mode': 'Single',
  'EV Comp': '0.0',
  'ISO MAX': '400',
  'ISO MIN': '200',
  'Megapixels': '12MP Linear',
  'Photo Sub Mode': 'Continuous',
  'Protune': 'ON',
  'RAW': 'OFF',
  'Sharpness': 'Low',
  'Shutter': '1/125',
  'WDR': 'ON',
  'White Balance': '3000K'},
 'Setup': {'Audio Input': 'Standard Mic',
  'Auto Off': '30 MIN',
  'Auto-Rotation': 'Auto',
  'Beeps': 'Low',
  'Current Flat Mode': 'Video',
  'Default Mode': 'Video',
  'GPS': 'ON',
  'LCD Brightness': '51',
  'LED': 'ON',
  'Language': 'English',
  'Quick Capture': 'OFF',
  'Screensaver': '1 MIN',
  'Secondary Stream Bit Rate': '2.5 Mbps',
  'Secondary Stream GOP Size': '8',
  'Secondary Stream IDR Interval': '1',
  'Secondary Stream Window Size': '480',
  'Video Format': 'NTSC',
  'Voice Control Enable': 'OFF',
  'Voice Control Language': 'English - US'},
 'Video': {'Color': 'GoPro',
  'Default Video Sub Mode': 'Video',
  'EV Comp': '0.0',
  'Field of View': 'Wide',
  'Fr